In [3]:
import scipy
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import numpy as np
import statistics
import math
import time
from tqdm.auto import trange, tqdm
import h5py
from itertools import product, zip_longest
import torch

/home/kimjosy/LocReg_Regularization-1/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
SNR = 10
n_elements = 128
d_start = 4
d_increment = 4

lambdaA = -6
lambdaB = 1
default_lambdas = np.append(0, np.logspace(lambdaA,lambdaB,51))

assert(np.all(default_lambdas>=0))

tdata = np.linspace(d_start, d_increment*n_elements, n_elements)
assert(tdata[0]==d_start)
assert(tdata[-1]==d_increment*n_elements)

############
#Test case to ensure everything is working
lambdas_comprehensive = np.append(0, np.logspace(lambdaA, lambdaB, 251))

#ground truth

c1 = 0.5
c2 = 0.5
T21 = 45
T22 = 200

ob_weight = 100
upper_bound = [1,1,500,1500] #Set upper bound on parameters c1, c2, T21, T22, respectively
true_params = np.array([c1, c2, T21, T22]) #Set initial guesses
initial_params = true_params
# initial_params = (0.5, 0.5, 250, 750) #Arbitrary params
agg_array = np.array([1,1,1/ob_weight, 1/ob_weight]) # This is to weigh the parameters equally...
# agg_array = np.array(1/initial_params)

round_Params = [round(num, 2) for num in true_params]
pList = ', '.join(str(x) for x in round_Params)

plt.style.use('default')

In [17]:
#Two parameter definition of s(t) with regularization parameter lambda

def G(t, con_1, con_2, tau_1, tau_2): 
    function = con_1*np.exp(-t/tau_1) + con_2*np.exp(-t/tau_2)
    return function

def G_tilde(lam, agg_arr = agg_array):
    #SA defines the signal amplitude, defaults to 1 for simulated data
    agg_arr = lam*agg_arr
    def Gt_lam(t, con1, con2, tau1, tau2):
        return np.append(G(t, con1, con2, tau1, tau2), [con1, con2, tau1,tau2]*agg_arr)
    return Gt_lam

def noise(sd):
    return np.random.normal(0, sd, n_elements)

def add_noise(signal, SNR):
    #Given a noiseless signal, adds noise at given SNR and returns a noisy signal
    signal_length = len(signal)
    noise_sd = signal[0]/SNR
    noisy_signal = signal + np.random.normal(0, noise_sd, signal_length)
    return noisy_signal

#Write your estimate_parameter func


In [32]:
lam = 0.01
noiseless_curve = G(tdata, c1, c2, T21, T22)
data = add_noise(noiseless_curve, SNR)
#estimate parameter func

(0.35236836566821333,
 0.6286367745343451,
 123.78576251451491,
 148.68151413253057)